In [17]:
from tqdm import tqdm
import sys
from numba import njit
import pandas as pd
from joblib import Parallel, delayed
import joblib
import contextlib

sys.set_int_max_str_digits(100000)

In [18]:
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""
    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()

In [19]:
def border_line_search():
    m = 1
    l = 1
    x = 0
    r = 0
    s = ''
    for i in tqdm(range(100000), total=100000):
        x0 = x
        r0 = r
        x1 = x+m
        r1 = r+l
        if r&1:
            l0 = 3*l
            l1 = l
            r0 = (3*r0+1)//2
            r1 //= 2
        else:
            l0 = l
            l1 = 3*l
            r0 //= 2
            r1 = (3*r1+1)//2
        m *= 2
        if (x0 == r0) or ((l0 < m) == (x0 < r0)):
            print(f'Try {m}n+{x0} =({i})=> {l0}n+{r0}')
        if (x1 == r1) or ((l1 < m) == (x1 < r1)):
            print(f'Try {m}n+{x1} =({i})=> {l1}n+{r1}')
        if m < l0 and l0 < 3*m:
            if m < l1 and l1 < 3*m:
                print(f'Impossible! {i} {m} {l0} {l1}')
                break
            x = x0
            r = r0
            s += '0' if l0 == l else '1'
            l = l0
        else:
            if l1 <= m or l1 >= 3*m:
                print(f'End of search space! {i} {m} {l0} {l1}')
                break
            x = x1
            r = r1
            s += '0' if l1 == l else '1'
            l = l1
        #print(f'Selected {m}n+{x} =({i})=> {l}n+{r}')
    #print(m, l, x, r, s)

In [20]:
#MM = 2**766512153894657
#LL = 3**483615324366283

#@njit
def get_upper_bound_for_min_cycle_element(i):
    m = 2**i
    n0 = 0
    j0 = 0
    for j in range(i):
        n = n0
        dn = 1
        if 3**j > m:
            break
        while m*n**j < (3*n+1)**j:
            #print(n, m*n**l, (3 * n + 1)**j)
            n += dn
            dn *= 2
        while dn > 1:
            dn //= 2
            n -= dn
            if m*n**j < (3*n+1)**j:
                n += dn
        if n0 < n:
            n0 = n
            j0 = j
    return j0, n0

def something(i, n):
    for x in range(3,n):
        r = x
        l = 1
        for j in range(i):
            if r&1:
                l *= 3
                r *= 3
                r += 1
            r //= 2
            if r < 3:
                break
        if r == x:
            print(i, x, r, m-l)

N = 100000
with tqdm_joblib(tqdm(desc="My calculation", total=N, mininterval=1)) as progress_bar:
    res = Parallel(n_jobs=10)(delayed(get_upper_bound_for_min_cycle_element)(i) for i in range(N))
pd.DataFrame(res, columns=["L", "N"]).to_csv('collatz_upper_bound_for_min_cycle_element.csv', index_label='M')


My calculation:  61%|██████    | 60697/100000 [29:34:19<39:18:08,  3.60s/it]     